# 2.5. Automatic Differentiation¶

autograd package is part of pytorch


In [1]:
from torch import tensor
import torch

"""
Assume we want to autograd function `y = 2x^Tx`
"""
x : tensor = torch.arange(4.0, requires_grad=True)
x


tensor([0., 1., 2., 3.], requires_grad=True)

In [2]:
x.grad # The gradient is None by default


In [3]:
y : tensor = 2 * torch.dot(x, x)
y


tensor(28., grad_fn=<MulBackward0>)

In [4]:
y.backward()
x.grad


tensor([ 0.,  4.,  8., 12.])

In [5]:
x.grad == 4 * x


tensor([True, True, True, True])

In [6]:
x.grad.zero_() # Reset gradient
y = x.sum()
y.backward()
x.grad


tensor([1., 1., 1., 1.])

## Backward for non-scalar variables


In [7]:
x, x.grad


(tensor([0., 1., 2., 3.], requires_grad=True), tensor([1., 1., 1., 1.]))

In [8]:
x.grad.zero_()
x, x.grad


(tensor([0., 1., 2., 3.], requires_grad=True), tensor([0., 0., 0., 0.]))

In [9]:
y = x * x
y.backward(gradient=torch.ones(len(y)))
x.grad


tensor([0., 2., 4., 6.])

## 2.5.3 Detaching computation

[http://d2l.ai/chapter_preliminaries/autograd.html#detaching-computation]()


In [10]:
x.grad.zero_()
x, x.grad


(tensor([0., 1., 2., 3.], requires_grad=True), tensor([0., 0., 0., 0.]))

In [11]:
y = x * x
u = y.detach()
z = u * x
u, z

(tensor([0., 1., 4., 9.]),
 tensor([ 0.,  1.,  8., 27.], grad_fn=<MulBackward0>))

In [12]:
z.sum().backward()
x.grad, x.grad == u


(tensor([0., 1., 4., 9.]), tensor([True, True, True, True]))

In [13]:
"""
Note that while this procedure detaches y’s ancestors from the graph leading to z, the computational graph leading to y persists and thus we can calculate the gradient of y with respect to x.
"""

x.grad.zero_()
x.grad


tensor([0., 0., 0., 0.])

In [14]:
y.sum().backward() # this is one-time operation! Once y is computed you can call .backward() only once!
x.grad, x.grad == 2 * x


(tensor([0., 2., 4., 6.]), tensor([True, True, True, True]))

## 2.5.4 Gradients and Python control flow

Automatic differentiation can still calculate the gradient of the resulting variable even if function involves complicated computations with branching, loops etc.

In [15]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c


In [16]:
a = torch.randn(size=(), requires_grad=True)
a


tensor(1.0539, requires_grad=True)

In [17]:
d = f(a)
d

tensor(1079.1482, grad_fn=<MulBackward0>)

In [18]:
d.backward()


In [19]:
a.grad, a.grad == d / a


(tensor(1024.), tensor(True))

## Exercises


In [20]:
a = torch.randn(size=(3, ), requires_grad=True)
d = f(a)
d.backward()
print(d, a, a.grad)



RuntimeError: grad can be implicitly created only for scalar outputs

In [ ]:
"""
This error occurs because by default, PyTorch's automatic differentiation framework assumes that the result of a computation is a scalar value. When calling backward(), PyTorch expects a scalar tensor to propagate gradients through the computational graph properly.

To overcome this issue and compute gradients for tensors with multiple elements, you need to specify a scalar quantity for backpropagation. This could involve taking the sum or mean of the tensor d before calling backward().
"""
a = torch.randn(size=(3, ), requires_grad=True)
d = f(a)
scalar_d = d.sum()  # Taking the sum to obtain a scalar quantity
scalar_d.backward()
print(d, scalar_d, a, a.grad)



In [ ]:
"""
Let f(x) =  sin(x). Plot the graph of f and of its derivative f'.
Do not exploit the fact that f' = cos(x), but rather use automatic differentiation to get the result.
"""


